In [4]:
!optimum-cli export onnx --model khanhld/wav2vec2-base-vietnamese-160h w2v2_onnx/

/home/spring/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/spring/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:872: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
		-[x] values not close enough, max diff: 8.33272933959961e-05 (atol: 1e-05)
The ONNX 

In [1]:
import torch
import librosa
import numpy as np
from jiwer import wer
import json
from transformers import Wav2Vec2Processor
from optimum.onnxruntime import ORTModelForCTC
from jiwer import wer 
import onnx 
from optimum.onnxruntime import ORTQuantizer, ORTModelForCTC
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from transformers import Wav2Vec2Processor

c:\Users\ADMIN\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

model = ORTModelForCTC.from_pretrained("w2v2_onnx/")
print(type(model))
processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")


<class 'optimum.onnxruntime.modeling_ort.ORTModelForCTC'>


In [4]:
#test model
def load_audio(file_path):
    speech_array, sampling_rate = librosa.load(file_path, sr=16000)
    inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True).input_values
    return inputs

def transcribe(file_path):
    input_values = load_audio(file_path)
    print(input_values.shape)
    logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_transcript = processor.batch_decode(pred_ids)[0]
    
    return pred_transcript



In [ ]:
preds = transcribe("wav/2.wav")
# print(f"predicted:{preds}")
with open('txt/2.txt', 'r') as file:
    true = file.read()
    # print(f"ground truth:{true}")
# print(f"WER:{wer(preds, true)}")

## Evaluate performance:

In [7]:
import time
def calculate_runtime():
    start = time.time()
    transcribe("wav/2.wav")
    end = time.time()
    print(f"Runtime: {end-start}")

## Quantize model

### Dynamic quantizing

In [10]:


processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")
quantizer = ORTQuantizer.from_pretrained(model)

save_dir = 'w2v2_quant'
quantization_config = AutoQuantizationConfig.avx2(is_static=False, per_channel=False)
model_quantized_path = quantizer.quantize(save_dir=save_dir, quantization_config=quantization_config)


In [16]:
#test quantized model
# print(model_quantized_path)
model = ORTModelForCTC.from_pretrained("w2v2_quant/")
print(type(model))
processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")


<class 'optimum.onnxruntime.modeling_ort.ORTModelForCTC'>


In [18]:
onnx_model_path ='w2v2_quant/model_quantized.onnx'
# Load the ONNX model to inspect the initializers
onnx_model = onnx.load(onnx_model_path)

# Initialize total size in bytes
total_size_bytes = 0

# Iterate over the initializers to calculate the size of each parameter
for initializer in onnx_model.graph.initializer:
    tensor_size = np.prod(initializer.dims)  # Calculate number of elements
    data_type = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[initializer.data_type]  # Get data type
    tensor_size_bytes = tensor_size * np.dtype(data_type).itemsize  # Calculate size in bytes
    total_size_bytes += tensor_size_bytes

print(f"Total size of quantized weights: {total_size_bytes / (1024**2):.2f} MB")

Total size of quantized weights: 90.43 MB


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3096\4091105623.py:11: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  data_type = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[initializer.data_type]  # Get data type


In [17]:
print(model)

optimized_model


### Evaluation

In [8]:
#evaluate quantized model
scores = []
runtimes = []
with open ('metadata.json', 'r') as file:
    metadata = json.load(file)
i = 0
for folder, transcript, filename in zip(metadata['folders'], metadata['transcripts'], metadata['filenames']):
    dest = f"vivos/train/waves/{folder}/{filename}.wav"
    transcript = transcript.lower()

    start = time.time()
    preds = transcribe(dest)
    end = time.time()
    runtime = end-start
    score = wer(preds, transcript)
    scores.append(score)
    
    print(f"transcript:{transcript}")
    print(f"predicted:{preds}")
    print(f"WER:{score}")
    print(f"Runtime: {runtime}")

    runtimes.append(runtime)
    i+=1
    if i==100:
        break
print(f"WER:{np.mean(scores)}")
print(f"Runtime: {np.mean(runtimes)}")

torch.Size([1, 48000])
transcript:khách sạn

predicted:hát sạn
WER:0.5
Runtime: 0.2689056396484375
torch.Size([1, 79000])
transcript:chỉ bằng cách luôn nỗ lực thì cuối cùng bạn mới được đền đáp

predicted:chị bằng cách lô nỗ lực thì cuối cùng bạn mới được đền đáp
WER:0.14285714285714285
Runtime: 0.3182411193847656
torch.Size([1, 69500])
transcript:trong số các quốc gia công nghiệp phát triển

predicted:trong số các quốc gia công nghiệp phát triển
WER:0.0
Runtime: 0.30170774459838867
torch.Size([1, 79500])
transcript:anh đã nhìn thấy trong những lải nhải dông dài của nhu

predicted:anh đã nhìn thấy trong những lại nhải dông dài của nhu
WER:0.08333333333333333
Runtime: 0.3381931781768799
torch.Size([1, 52000])
transcript:khủng hoảng môi trường cần được ngăn chặn

predicted:khủng hoàng môi trường cần được ngăn crặn
WER:0.25
Runtime: 0.23973679542541504
torch.Size([1, 72000])
transcript:trông thấy tàu cá đằng xa anh đã ấn còi cảnh báo

predicted:trong thoấy tàu cá đằng xa anh đã ấn còi cản

### Static quantize

In [4]:
from optimum.onnxruntime.configuration import AutoCalibrationConfig, AutoQuantizationConfig
from datasets import Dataset
onnx_model_path = 'w2v2_quant_statics'
quantizer = ORTQuantizer.from_pretrained(model)

qconfig = AutoQuantizationConfig.avx2(is_static=True, per_channel=False)
dataset = {}
dataset['input_values'] = []

with open ('metadata.json', 'r') as file:
    metadata = json.load(file)
i = 0
for folder, transcript, filename in zip(metadata['folders'], metadata['transcripts'], metadata['filenames']):
    dest = f"vivos/train/waves/{folder}/{filename}.wav"
    audio, sr = librosa.load(dest, sr=16000)
    dataset['input_values'].append(audio)
    i+=1
    if i>=100:
        break

calibration_dataset = Dataset.from_dict(dataset)

calibration_config = AutoCalibrationConfig.minmax(calibration_dataset)

ranges = quantizer.fit(dataset=calibration_dataset, 
                       calibration_config=calibration_config,
                       operators_to_quantize= qconfig.operators_to_quantize,)

model_quantized_path = quantizer.quantize(save_dir=onnx_model_path, 
                                          calibration_tensors_range=ranges,
                                          quantization_config=qconfig,)

print("Done")

Done


In [ ]:
onnx_model_path ='w2v2_quant_statics/model_quantized.onnx'
# Load the ONNX model to inspect the initializers
onnx_model = onnx.load(onnx_model_path)

# Initialize total size in bytes
total_size_bytes = 0
total_params = 0
# Iterate over the initializers to calculate the size of each parameter
for initializer in onnx_model.graph.initializer:
    tensor_size = np.prod(initializer.dims)
    total_params += tensor_size # Calculate number of elements
    data_type = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[initializer.data_type]
    tensor_size_bytes = tensor_size * np.dtype(data_type).itemsize  # Calculate size in bytes
    total_size_bytes += tensor_size_bytes

print(f"Total size of quantized weights: {total_size_bytes / (1024**2):.2f} MB")
print(f"Total number of parameters: {total_params}")

float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5472\3862554855.py:12: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  data_type = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[initializer.data_type]


In [16]:
folders = []
transcripts = []
filenames = []
with open(f'vivos/test/prompts.txt', 'r') as f:
    for line in f:
        filename, transcript = line.split(' ', 1)
        folder = filename.split('_')[0]
        # print(f"folder: {folder}, transcript: {transcript}, filename: {filename}")
        folders.append(folder)
        transcripts.append(transcript)
        filenames.append(filename)

metadata = {'folders': folders, 'transcripts': transcripts, 'filenames': filenames}
with open('metadata_test.json', 'w') as f:
    json.dump(metadata, f)
    

### Evaluation

In [12]:
#evaluate quantized model
scores = []
runtimes = []
with open ('metadata_test.json', 'r') as file:
    metadata = json.load(file)
i = 0
for folder, transcript, filename in zip(metadata['folders'], metadata['transcripts'], metadata['filenames']):
    dest = f"vivos/test/waves/{folder}/{filename}.wav"
    start = time.time()
    preds = transcribe(dest)
    end = time.time()
    runtime = end-start

    transcript = transcript.lower()  
    score = wer(preds, transcript)
    scores.append(score)
    runtimes.append(runtime)

    print(f"transcript:{transcript}")
    print(f"predicted:{preds}")
    print(f"WER:{score}")
    i+=1
    if i==100:
        break

print(f"Average WER:{np.mean(scores)}")
print(f"Average Runtime: {np.mean(runtimes)}")

torch.Size([1, 42000])
transcript:trở nên thụ động

predicted:trợ nên thụ động
WER:0.25
torch.Size([1, 41000])
transcript:cũng khiến cho họ dè dặt

predicted:cũng khiến cho họ dề dặt
WER:0.16666666666666666
torch.Size([1, 70500])
transcript:chị gặn hỏi anh thề sống thề chết là không có

predicted:chị gặng hỏi anh thể sóng thể chết đã không có
WER:0.45454545454545453
torch.Size([1, 62000])
transcript:điện thoại reng nhưng ta cũng phải nhấc máy đúng không

predicted:tiện thại len nhưng ta cũng phải nhất máy đúng không
WER:0.36363636363636365
torch.Size([1, 59500])
transcript:cũng thuộc loại dày đặc nhất trong hệ mặt trời

predicted:cốn thộ lại gày đạt nhất trong hệ mặt trời
WER:0.5
torch.Size([1, 57000])
transcript:các vụ tham nhũng và bê bối kinh tế

predicted:các vụ tham nhũn và bê bối kinh tế
WER:0.1111111111111111
torch.Size([1, 76000])
transcript:chúng cứ cố lùi lại mặc dù vướng víu gông xiềng

predicted:chúng cứ cố lùi lại mặch dồ bướng phiếu gông triền
WER:0.45454545454545453
torc

In [1]:
print(model.base_model_prefix)

NameError: name 'model' is not defined

## Give TVM a shot cuz why not? (NVM)

In [21]:
import onnx
import numpy as np
import tvm.relay as relay
import tvm
from tvm.contrib import graph_executor
import librosa
#load w2v2_onnx model
model_path = "w2v2_onnx/model.onnx"
onnx_model = onnx.load(model_path)



In [23]:
target = "llvm"
input_name = 'input'
input_data = load_audio("wav/2.wav")
shape_dict = {input_name: [0,0]}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

/home/spring/tvm/python/tvm/relay/frontend/onnx.py:7234: UserWarning: 'ORTModelForCTC' object has no attribute 'SerializeToString'
  warnings.warn(str(e))


AttributeError: 'ORTModelForCTC' object has no attribute 'graph'

In [16]:
import onnx

# Load your ONNX model
onnx_model = onnx.load(model_path)

# Print the input names
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(f"Input shape: {[dim.dim_value for dim in input.type.tensor_type.shape.dim]}")

Input name: input_values
Input shape: [0, 0]
